In [ ]:
import pandas as pd


def calculate_synthetic_atm(strike_price):
    return round(strike_price / 100) * 100


def select_straddle_range(synthetic_atm):
    return [synthetic_atm - 200, synthetic_atm - 100, synthetic_atm, synthetic_atm + 100, synthetic_atm + 200]


def process_strategy_data(input_csv, output_excel):
   
    data = pd.read_csv(input_csv)
    data['datetime'] = pd.to_datetime(data['datetime'])
    
    
    data['SyntheticATM'] = data['strike_price'].apply(calculate_synthetic_atm)
    data['StraddleRange'] = data['SyntheticATM'].apply(select_straddle_range)
    
   
    data['Entry'] = data['close'] < data['VWAP'].fillna(0)  
    data['Profit'] = data['close'] - data['open']           
    
    
    summary = data.groupby('SyntheticATM').agg(
        Total_Trades=('Entry', 'sum'),
        Total_Net_Profit=('Profit', 'sum')
    ).reset_index()
    
    
    summary['Range Name'] = summary['SyntheticATM'].apply(lambda x: f"ATM_{x}")
    summary['SpotTime'] = 918  
    summary['SL Threshold'] = 100  
    summary['DTE'] = 0  
    summary['Strategy_Name'] = "23_EMA_High"
    summary['Raange_Category'] = "T1_100"
    summary['TimeFrame'] = "3min"
    
    
    output_columns = [
        'Range Name', 'Total_Trades', 'Total_Net_Profit', 'SpotTime', 
        'SL Threshold', 'DTE', 'Strategy_Name', 'Raange_Category', 'TimeFrame'
    ]
    formatted_output = summary[output_columns]
    
    
    formatted_output.to_excel(output_excel, index=False)
    print(f"Output saved to: {output_excel}")


input_csv = 'E:/Quant Intern/Nifty Data/NIFTY_2023-08-10.csv'
output_excel = 'E:/Quant Intern/Output folder/strategy_results_output.xlsx'


process_strategy_data(input_csv, output_excel)
